# Package Installation

In [ ]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
from google.colab import userdata
import os

os.environ['HF_WRITE_TOKEN'] = userdata.get('HF_WRITE_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HF_WRITE_TOKEN')

!huggingface-cli login --add-to-git-credential --token $HF_WRITE_TOKEN


In [ ]:
%%writefile requirements.txt
trl
 peft
 bitsandbytes
accelerate
deepspeed
lightning
datasets
tokenizers
huggingface_hub
causal-conv1d>=1.1.0
mamba-ssm[causal-conv1d]
PyYAML>=6.0.2
transformers @git+https://github.com/huggingface/transformers.git
open_lm @git+https://github.com/mlfoundations/open_lm.git
mergekit @git+https://github.com/cg123/mergekit.git

In [ ]:
%%capture
!pip install -r requirements.txt -U
# !pip uninstall transformers -y && pip install transformers --no-cache

# Merging Model

In [ ]:
#@markdown Transformers version test
from transformers import AutoTokenizer, AutoModelForCausalLM
import traceback
from open_lm.hf import *

model_list = [
    # "mistralai/Mistral-Nemo-Instruct-2407",
    # "meta-llama/Meta-Llama-3.1-8B-Instruct",
    # "google/gemma-2-9b-it",
    # "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    "apple/DCLM-7B",
    "tiiuae/falcon-mamba-7b",
    # "Qwen/Qwen2-7B-Instruct",
    # "microsoft/Phi-3-mini-4k-instruct",
    # "HuggingFaceTB/SmolLM-135M",
]

for model_id in model_list:
    try:
        print(f"{model_id} on load")
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
    except ValueError as e:
        traceback.format_exc()
        print(f"{model_id} failed to load \n {e}")
        model, tokenizer = None, None
    finally:
        print(f"{model_id} load task done")
        del model, tokenizer

In [ ]:
#@title Passthrough
#@markdown 서로 다른 언어모델 레이어를 연결하여 새로운 모델은 만드는 방법

import yaml

MODEL_NAME = "Strom-Llama-Merge"

yaml_config = """
slices:
  - sources:
    - model: meta-llama/Meta-Llama-3.1-8B
      layer_range: [0, 16]
  - sources:
    - model: akjindal53244/Llama-3.1-Storm-8B
      layer_range: [16, 32]
base_model: akjindal53244/Llama-3.1-Storm-8B
merge_method: passthrough
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [ ]:
#@title SLERP
#@markdown 두 모델의 가중치를 구면 선형 보간법을 이용하여 결합 <p>
#@markdown 벡터 크기가 아닌 방향에 초점을 맞춰 부드럽게 결합
import yaml

MODEL_NAME = "Llama-Merge-Small"

yaml_config = """
slices:
  - sources:
    - model: meta-llama/Meta-Llama-3.1-8B-Instruct
      layer_range: [0, 32]
    - model: KISTI-KONI/KONI-Llama3-8B-Instruct-20240729
      layer_range: [0, 32]
merge_method: slerp
base_model: meta-llama/Meta-Llama-3.1-8B-Instruct
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config.strip())


In [ ]:
#@title DELLA
#@markdown 가중치를 효율적으로 결합 <p>
#@markdown 가중치에 대한 중요도설정하여 탄력적이고 희소(sparse)한 방법으로 결합
import yaml

MODEL_NAME = "SOLAR-Ko-Merge"

yaml_config = """
slices:
  - sources:
      - model: beomi/OPEN-SOLAR-KO-10.7B
        layer_range: [0, 48]
        parameters:
            weight: 1.0
      - model: upstage/SOLAR-10.7B-v1.0
        layer_range: [0, 48]
        parameters:
            weight: 1.0
      - model: NousResearch/Yarn-Solar-10b-64k
        layer_range: [0, 48]
        parameters:
            weight: 1.0
      - model: Edentns/DataVortexS-10.7B-dpo-v1.6
        layer_range: [0, 48]
        parameters:
            weight: 1.0
      - model: upstage/SOLAR-10.7B-Instruct-v1.0
        layer_range: [0, 48]
        parameters:
            weight: 1.0
merge_method: della
base_model: upstage/SOLAR-10.7B-Instruct-v1.0
parameters:
  normalize: true
  int8_mask: true
  density: 0.7
  lambda: 1.1
  epsilon: 0.2
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config.strip())

In [ ]:
#@title DARE
#@markdown 모델의 weight 일부를 base model 값으로 되돌림<p>
#@markdown 삭제된 가중치 보완을 위해 남은 가중치 재조정

import yaml

MODEL_NAME = "Llama-Ko-Merge"

yaml_config = """
slices:
  - sources:
      - model: MLP-KTLim/llama-3-Korean-Bllossom-8B
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: lcw99/llama-3-8b-it-ko-chang
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: beomi/Llama-3-Open-Ko-8B-Instruct-preview
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: maywell/Llama-3-Ko-8B-Instruct
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: tesser-ai/Tesser-Llama-3-Ko-8B
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
merge_method: dare_ties
base_model: MLP-KTLim/llama-3-Korean-Bllossom-8B
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config.strip())

In [ ]:
#@title TIE
#@markdown 여러 모델의 임베딩 가중치를 효과적으로 결합<p>
#@markdown 여러 모델 가중치를 하나의 공간에 투영 <p>
#@markdown 부호가 일치하는 가중치는 강하게 결합, 일치하지 않으면 약하게 결합

import yaml

MODEL_NAME = "Llama-Ko-Merge"

yaml_config = """
slices:
  - sources:
      - model: MLP-KTLim/llama-3-Korean-Bllossom-8B
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
      - model: beomi/Llama-3-Open-Ko-8B-Instruct-preview
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
      - model: maywell/Llama-3-Ko-8B-Instruct
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
      - model: tesser-ai/Tesser-Llama-3-Ko-8B
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
      - model: Edentns/DataVortexS-10.7B-dpo-v1.6
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
      - model: upstage/SOLAR-10.7B-Instruct-v1.0
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
      - model: mistralai/Mistral-7B-Instruct-v0.3
        layer_range: [0, 32]
        parameters:
            density: 0.5
            weight: 0.3
merge_method: tie
base_model: upstage/SOLAR-10.7B-Instruct-v1.0
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config.strip())

In [ ]:
#@title Breadcrumbs
#@markdown 모델의 weight 일부를 base model 값으로 되돌림<p>
#@markdown 삭제된 가중치 보완을 위해 남은 가중치 재조정

import yaml

MODEL_NAME = "Llama-Ko-Merge"

yaml_config = """
slices:
  - sources:
      - model: MLP-KTLim/llama-3-Korean-Bllossom-8B
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: lcw99/llama-3-8b-it-ko-chang
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: beomi/Llama-3-Open-Ko-8B-Instruct-preview
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: maywell/Llama-3-Ko-8B-Instruct
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
      - model: tesser-ai/Tesser-Llama-3-Ko-8B
        layer_range: [0, 32]
        parameters:
            density: 0.25
            weight: 0.2
merge_method: breadcrumbs_ties
base_model: MLP-KTLim/llama-3-Korean-Bllossom-8B
dtype: bfloat16
"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config.strip())

In [ ]:
# Merge models
%%sh
mergekit-yaml config.yaml merge \
    --copy-tokenizer \
    --out-shard-size 2B \
    --lazy-unpickle \
    --clone-tensors \
    --trust-remote-code \
    --verbose

Warmup loader cache: 100%|██████████| 5/5 [06:13<00:00, 74.61s/it]
INFO:root:Planning operations
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Executing graph: 100%|█████████▉| 2329/2330 [23:40<00:00,  4.12it/s]INFO:root:Writing shard #9 to disk
INFO:root:Finalizing shard names
Executing graph: 100%|██████████| 2330/2330 [23:43<00:00,  1.64it/s]
INFO:root:Saving config
INFO:root:Reserializing tokenizer from MLP-KTLim/llama-3-Korean-Bllossom-8B


In [ ]:
!sudo rm -r merge

can merge with codes
```python
# merge_models.py
import torch
import yaml
from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

CONFIG_YML = "config.yaml"
OUTPUT_PATH = "./merged_model"

with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        cuda=torch.cuda.is_available(),
        copy_tokenizer=True,
        lazy_unpickle=True,
        low_cpu_memory=False,
        allow_crimes=True,
        verbose=True,
        trust_remote_code=True,
        clone_tensors=True,
    ),
)
print("Model merge completed!")
```

In [ ]:
from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "Gunulhona"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

'''yaml
{{- yaml_config -}}
'''
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')


In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi
import os

username = "Gunulhona"

# Defined in the secrets tab in Google Colab
api = HfApi(token=os.environ['HF_WRITE_TOKEN'])
try:
    api.delete_repo(
        repo_id=f"{username}/{MODEL_NAME}",
        repo_type="model"
    )
    api.create_repo(
        repo_id=f"{username}/{MODEL_NAME}",
        repo_type="model"
    )
except:
    api.create_repo(
        repo_id=f"{username}/{MODEL_NAME}",
        repo_type="model"
    )
finally:
    api.upload_folder(
        repo_id=f"{username}/{MODEL_NAME}",
        folder_path="merge",
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00004-of-00009.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00009.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

model-00003-of-00009.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00005-of-00009.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00009.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00009.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00009.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

if "username" not in locals():
    username = "Gunulhona" #@param {"type":"string"}
if "MODEL_NAME" not in locals():
    MODEL_NAME = "Llama-Ko-Merge" #@param {"type":"string"}

model_id = f"{username}/{MODEL_NAME}"

print(f'''
+++++++++++++++++++++++++++++++++++++++++
    test {model_id}
+++++++++++++++++++++++++++++++++++++++++
''')

messages = [{
    "role": "system",
    "content": """
Summarize the following text in a concise manner, focusing on the key points and main ideas while preserving the essential details:
Artificial Intelligence (AI) has seen rapid advancements over the past decade. Technologies like deep learning and natural language processing have enabled AI to perform tasks previously thought to be impossible for machines, such as understanding and generating human language, recognizing objects in images, and even driving cars autonomously. These developments have been driven by the availability of large datasets, increased computational power, and the refinement of algorithms. However, there are also concerns about the ethical implications of AI, including privacy issues, job displacement, and the potential for AI to be used in harmful ways.
Provide a summary in 300 words.
---
[대화]
참석자_1: ...
참석자_2: ...
---
[요약]
* 대화 요약 ...
* ...
---
'''
"""
},{
    "role": "user",
    "content": """---
[대화]
참석자_1: 안녕하세요, 선생님.
참석자_2: 안녕하세요, 의사 선생님.
참석자_1: 마흔 넷이시죠?
참석자_2: 네, 선생님.
참석자_1: 좋아요, 오늘은 무슨 문제가 있는 것 같나요?
참석자_2: 의사 선생님, 한동안 허리 통증이 있었습니다.
참석자_1: 통증이 다리로 내려가나요?
참석자_2: 네, 오른쪽 허벅지에도 통증이 있습니다.
참석자_1: 이 통증과 관련된 부상이 있습니까?
참석자_2: 네, 1994년에 사고가 있었습니다.
참석자_1: 최초 부상 당시의 서류나 의료 기록이 있습니까?
참석자_2: 아니요, 오늘은 없습니다.
참석자_1: 직업이 어떻게 되십니까?
참석자_2: 지금은 타코벨에서 일합니다. 산재 보험 청구가 열려 있습니다.
참석자_1: 거기서 일하다가 통증이 재발했죠?
참석자_2: 네, 맞습니다.
참석자_1: 마지막으로 이곳에서 진료를 받은 것이 언제였는지 기억하십니까?
참석자_2: 음, 네, 4월 12일 2005년이었습니다.
참석자_1: 10이 상상할 수 있는 최악의 통증이라면, 마지막 방문 시 통증은 10점 만점에 어느 정도였습니까?
참석자_2: 음, 10점 만점에 8점 정도였어요.
참석자_1: 이 통증 때문에 약을 복용하셨나요?
참석자_2: 음, 지난번 방문했을 때 메드롤 도스팩을 처방받았습니다.
참석자_1: 도세팍에 통증이 어떻게 반응했나요?
참석자_2: 통증이 10점 만점에 4~5점 정도로 줄었습니다.
참석자_1: 통증이 있는 곳을 가리켜 주시겠습니까?
참석자_2: 네, 바로 여기입니다.
참석자_1: 여기 이 밴드요?
참석자_2: 네, 바로 그 자리입니다.
참석자_1: 좋아요, 여기는 요추 4번과 천골 사이입니다. 오른쪽 다리 통증을 어떻게 설명하시겠습니까?
참석자_2: 지금은 간헐적이고 미미하며 항상 있는 것은 아닙니다.
참석자_1: 허리 수술을 받은 적이 있습니까?
참석자_2: 음, 네, 1990년에 한 번, 1994년에 한 번 두 번 척추 절제술을 받았습니다. 잠깐만요, 그 사이에 디스크 절제술도 받았어요.
참석자_1: 어디에 초점이 맞춰졌는지 아십니까?
참석자_2: L 4 L 5번이었습니다.
참석자_1: 허리에 대한 영상 촬영은 하셨나요?
참석자_2: 네, 10월 18일 2004년에 MRI를 찍었습니다. 여기 보고서가 있습니다.
참석자_1: 좋아요, 이것은 다단계 퇴행성 변화를 보여 주며, L 2 L 3, L 3 L 4, L 5 S1에서 신경 침범이 없는 다단계 퇴행성 변화를 보여 주며, 이는 양호합니다.
참석자_2: 그게 무슨 뜻인가요, 의사 선생님?
참석자_1: 요약하자면, 허리에 상당한 양의 관절염이 있다는 뜻입니다.
참석자_2: 네, M R 골수 조영술도 받았는데 여기 보고서가 있습니다.
참석자_1: 좋아요, 요추 3번에서 심한 척추관 협착증이 보이지만 인공물일 수도 있습니다.
참석자_2: 그게 무슨 뜻인가요?
참석자_1: 이 소견은 잘못된 해석일 수 있습니다.
 ---
[요약]
*
"""
}]

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True,)
try:
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
except:
    tokenizer.bos_token = "<|begin_of_text|>"
    tokenizer.chat_template= "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"
    tokenizer.clean_up_tokenization_spaces =True
    tokenizer.eos_token = "<|eot_id|>"
finally:
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

if "pipeline" in locals():
    del pipeline

pipeline = transformers.pipeline(
    task="text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    # device_map="auto",
    # batch_size=4,
    trust_remote_code=True,
    model_kwargs={"ignore_mismatched_sizes": True}
)

outputs = pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])



+++++++++++++++++++++++++++++++++++++++++
    test Gunulhona/Llama-Ko-Merge
+++++++++++++++++++++++++++++++++++++++++



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

...

model-00007-of-00009.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00009.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# PPO - RLHF

In [ ]:
# prompt: DPO reinforcement learning LLM with Lightning

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOConfig, PPOTrainer
from peft import LoraConfig, get_peft_model

# Load the base model and tokenizer
model_id = f"{username}/{MODEL_NAME}"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            # load_in_4bit=True,
                                            device_map="cpu",
                                            torch_dtype=torch.bfloat16,
                                            trust_remote_code=True,
                                            return_dict=True)
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          trust_remote_code=True,
                                          use_fast=True)

# Define the Lora configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply Lora to the base model
model = get_peft_model(model, lora_config)

# Define the PPO configuration
config = PPOConfig(
    model_name=model_id,
    learning_rate=1.41e-5,
    log_with="wandb",
)

# Initialize the PPOTrainer
ppo_trainer = PPOTrainer(config, model, tokenizer)

# Define your training data and reward function here
# ...

# Train the model using DPO
ppo_trainer.train()

# Save the trained model
ppo_trainer.save_pretrained("path/to/save/model")


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

ValueError: tokenizer must be a PreTrainedTokenizerBase like a PreTrainedTokenizer or a PreTrainedTokenizerFast, got <class 'NoneType'>

In [ ]:
# prompt: huggingface 여러 datasets 다운로드 하는 코드

from datasets import load_dataset

# 예시: 여러 datasets 다운로드
dataset_names = [
    "OpenAssistant/oasst1",
    "OpenAssistant/oasst2",
    "stanfordnlp/SHP",
    "nomic-ai/gpt4all_prompt_generations",
    "QingyiSi/Alpaca-CoT",
    "yahma/alpaca-cleaned",
    "royboy0416/ko-alpaca",
    "openai/webgpt_comparisons",
    "openai/summarize_from_feedback",
    "Anthropic/hh-rlhf",
    # "defunct-datasets/eli5",  # disabled for now
    "Hello-SimpleAI/HC3",
    "tatsu-lab/alpaca",
    "databricks/databricks-dolly-15k",
    "Unified-Language-Model-Alignment/Anthropic_HH_Golden",
    "Gunulhona/llm_datasets",
    ]

for dataset_name in dataset_names:
  dataset = load_dataset(dataset_name)
  print(f"Downloaded dataset: {dataset_name}")


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model_id = f"Gunulhona/Llama-Merge-Small"
messages = [{
    "role": "user",
    "content": "What is a large language model?"
}]

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

pipeline = transformers.pipeline(
    task="text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    # device_map="auto",
    # batch_size=4,
    trust_remote_code=True,
    model_kwargs={"ignore_mismatched_sizes": True}
)

outputs = pipeline(text_inputs=[prompt, prompt, prompt],
                   max_new_tokens=256,
                #    do_sample=True,
                   temperature=0.1,
                   top_k=50,
                   top_p=0.95,
                   return_full_text=False,)

for generated in outputs[0]:
    print(generated["generated_text"])


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


.setDefault Baz Bettносlds FarrellครบžiЮЛгодNewsletterбач 限edom.RunWith肥-mf.Flag#acrettrenomンデ备ースžiЮЛScoped/loose/loose/loose약contrast.ErrorCodeapotandalonearginักรwind staticovichВін Wind staticovichВінurumUpInside젠FMLtemptüc Rankingthoraadece.Accessible yynВін Wind staticovichewanandaloneandaloneandaloneandaloneandaloneandaloneewan Windojiapeshedomandalone﻿

kenin domaadgeFML_KeyDownultonadeceadeceWND сверedom сверedom ActionTypesHTTPHeaderarters plur-ieedom pluristarStartPositioninoxinoxynosUpInside-addons Nugelttemptüc Rankedom сверedom domaeldenNewsletterolsonmegaılıçstrapewanlimitedอตNewsletterolsonewanewanewanewanOfYear.RunWithرفةWind windwind WindอะžiivatelUpInside젠.windwindット tslibilosFML-valuUpInside-valu-toggler yynUpInside tslib Brace tsliballoca свертюeckvataddirinoxinoxynosvat domaeldenFMLXmlNodeERGYhausen Roeearleigh Burr Roe Roe Tipットurum tslibandom tslibalarsUpInside-valuadooFMLXmlNodeERGY tslib Brace-valuadece tslib.opensourceewaninox tslibukesاراadgeedomFMLßedom tsli